In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score, confusion_matrix
from sklearn.metrics import silhouette_score
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from dateutil.relativedelta import relativedelta
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel

# Data Manipulation & Engineering new features

In [2]:
df=pd.read_csv('customer_revenue_tier_price.csv').drop('Unnamed: 0',axis=1)
#cx_case=pd.read_csv('cx_cases.csv')
cx_interact=pd.read_csv('cx_interactions_0625.csv')
site_install=pd.read_csv('site_install_0722.csv')
svc_incident = pd.read_csv('svc_incidents_0625.csv')

In [3]:
df=df.rename(columns={'Active':'Active_20190601'})

In [4]:
def multiple_item(x):
    if len(x[0])>1:
        return "Multiple"
    else:
        return x

In [5]:
def LatestAmtRatio(customer_id,duration,trx_max,frequency,monetary,data_end):
    if frequency <= 2:
        return 1
    else:
        if duration <= 366:
            return 1
        else:
            k = round(duration/3)
            delta=pd.to_timedelta('{} days'.format(k))
            cal_start_date = trx_max-delta
            ## using the df
            latest_amt = df[(df['CUSTOMER_SITE_ID']==customer_id)&\
                            (df['TRX_DATE']<=data_end)&\
                            (df['TRX_DATE']>=cal_start_date)]['TRX_AMT_USD'].sum()
            latest_ratio = (latest_amt*3)/monetary
            return latest_ratio

In [6]:
def LatestPriceRatioTrend(customer_id,duration,trx_max,frequency,agg_price_index,data_end):
    if frequency <= 2:
        return 1
    else:
        if duration <= 366:
            return 1
        else:
            k = round(duration/3)
            delta=pd.to_timedelta('{} days'.format(k))
            cal_start_date = trx_max-delta
            ## using the df
            latest_ws = df[(df['CUSTOMER_SITE_ID']==customer_id)&\
                           (df['TRX_DATE']<=data_end)&\
                           (df['TRX_DATE']>=cal_start_date)]['Price_WS_proportion'].sum()
            latest_stu = df[(df['CUSTOMER_SITE_ID']==customer_id)&\
                            (df['TRX_DATE']<=data_end)&\
                            (df['TRX_DATE']>=cal_start_date)]['Price_STU_proportion'].sum()
            latest_price_ratio= latest_ws/latest_stu
            latest_price_ratio_trend = latest_price_ratio/agg_price_index
            return latest_price_ratio_trend

In [7]:
def DfTimeFeature(df,data_start,data_end,svc_incident,site_install,cx_interact):
    df['TRX_DATE']=pd.to_datetime(df['TRX_DATE'])
    data_start =pd.to_datetime(data_start)
    data_end =pd.to_datetime(data_end)
    
    df['Main_Product']=np.where(df['PRODUCT_FAMILY'].isin(['CIJ','LCM','TTO','TIJ']),df['PRODUCT_FAMILY'],'Other')
    df['Qty_proportion']=df['QUANTITY']/df['Qty']
    df['Price_WS_proportion']=df['Site_Level_Price_Index_WS']*df['Qty_proportion']
    df['Price_STU_proportion']=df['Site_Level_Price_Index_STU']*df['Qty_proportion']
    
    df_time = df[(df['TRX_DATE']>=data_start)&(df['TRX_DATE']<=data_end)]
    

    #aggrefate on customer_site_id level
    dfg=df_time.groupby(['CUSTOMER_SITE_ID','Tier',\
                      'SHORT_VERTICAL','CUSTOMER_CLASS'])\
    .agg({"SALES_CHANNEL":pd.Series.mode,'PRODUCT_MODEL':pd.Series.mode,'Main_Product':pd.Series.mode,\
     'TRX_DATE':['min','max'],'TRX_AMT_USD':'sum','CUSTOMER_TRX_ID':pd.Series.nunique,\
        "ITEM_ID":pd.Series.nunique,'Price_WS_proportion':'sum','Price_STU_proportion':'sum'}).reset_index()

    dfg.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in dfg.columns]
    
    #get feature from the join df
    dfg=dfg.rename(columns={'TRX_AMT_USD_sum':'Monetary','CUSTOMER_TRX_ID_nunique':'Frequency',\
                     'ITEM_ID_nunique':'Unique_Item_Purchased'})
    dfg['Aggregate_index']=dfg['Price_WS_proportion_sum']/dfg['Price_STU_proportion_sum']
    dfg['Duration']=((dfg['TRX_DATE_max']-dfg['TRX_DATE_min'])/np.timedelta64(1,'D')).astype('int')+1
    dfg['Recency']=(((data_end-dfg['TRX_DATE_max'])/np.timedelta64(1,'D')).astype('int')+1)
    dfg['SALES_CHANNEL_mode']=dfg['SALES_CHANNEL_mode'].apply(multiple_item)
    dfg['PRODUCT_MODEL_mode']=dfg['PRODUCT_MODEL_mode'].apply(multiple_item)
    dfg['Main_Product_mode']=dfg['Main_Product_mode'].apply(multiple_item)
    dfg['Avg_AMT']=dfg['Monetary']/dfg['Frequency']
    dfg['Avg_Duration']=dfg['Duration']/dfg['Frequency']
    dfg_with_dummy = pd.get_dummies(dfg, columns=['SHORT_VERTICAL', 'CUSTOMER_CLASS',
                                               'SALES_CHANNEL_mode', 'PRODUCT_MODEL_mode','Main_Product_mode'])
    dfg_with_dummy['Lastest_Amt_Ratio']=dfg_with_dummy.apply(lambda x: LatestAmtRatio(x.CUSTOMER_SITE_ID, \
                                     x.Duration,x.TRX_DATE_max,x.Frequency,x.Monetary,data_end), axis=1)
    dfg_with_dummy['Lastest_Price_Ratio_Trend']=dfg_with_dummy.apply(lambda x: LatestPriceRatioTrend(x.CUSTOMER_SITE_ID, \
                                     x.Duration,x.TRX_DATE_max,x.Frequency,x.Aggregate_index,data_end), axis=1)
    
    #get feature from svc_incident
    svc_incident['LAST_SERVICE_EVENT_DATE']=pd.to_datetime(svc_incident['LAST_SERVICE_EVENT_DATE'])
    svc_time = svc_incident[(svc_incident['LAST_SERVICE_EVENT_DATE']>=data_start)&\
                               (svc_incident['LAST_SERVICE_EVENT_DATE']<=data_end)]
    incident_feature=svc_time.groupby('CUSTOMER_SITE_ID').agg({'NO_OF_SVC_INCIDENTS':'sum'}).reset_index()
    
    site_install['INSTANCE_LAST_UPDATE_DATE']=pd.to_datetime(site_install['INSTANCE_LAST_UPDATE_DATE'])
    site_time = site_install[(site_install['INSTANCE_LAST_UPDATE_DATE']<=data_end)]
    site_time['Contract']=np.where(site_time['CONTRACT_FLAG']=='Y',1,0)
    site_feature=site_time.groupby('CUSTOMER_SITE_ID').agg({"INSTANCE_ID":pd.Series.nunique,\
                                                           "Contract":'sum'}).reset_index()
    
    id_with_incident=dfg_with_dummy.merge(incident_feature,on='CUSTOMER_SITE_ID')['CUSTOMER_SITE_ID'].values
    df_no_incident=dfg_with_dummy[~(dfg_with_dummy['CUSTOMER_SITE_ID'].isin(id_with_incident))]
    df_no_incident['NO_OF_SVC_INCIDENTS']=0
    #some customer don't have incident but stil have contracted printer
    df_no_incident2=df_no_incident.merge(site_feature[['CUSTOMER_SITE_ID','Contract']],on='CUSTOMER_SITE_ID',how='left')
    df_no_incident2['Contract']=df_no_incident2['Contract'].fillna(0)
    df_no_incident2['Incident_per_printer']=0
    
    df_incident=dfg_with_dummy.merge(incident_feature,on='CUSTOMER_SITE_ID').merge(site_feature,on='CUSTOMER_SITE_ID')
    df_incident['Incident_per_printer']=df_incident['NO_OF_SVC_INCIDENTS']/df_incident['INSTANCE_ID']
    df_incident=df_incident.drop('INSTANCE_ID',axis=1)
    
    df_feature=pd.concat([df_incident,df_no_incident2])
    
    #get feature from cx interact
    cx_interact['Created_Date']=pd.to_datetime(cx_interact['Created_Date'])
    cx_time= cx_interact[(cx_interact['Created_Date']>=data_start)&\
                        (cx_interact['Created_Date']<=data_end)]
    interact_feature=cx_time.groupby('CUSTOMER_SITE_ID').agg({'Visit':'sum'}).reset_index()
    df_feature2=df_feature.merge(interact_feature,on='CUSTOMER_SITE_ID',how='left')
    df_feature2['Visit']=df_feature2['Visit'].fillna(0)
    df_feature2['Visit_per_TRX']=df_feature2['Visit']/df_feature2['Frequency']
    
    return df_feature2

In [8]:
data_start_18='2015-01-01'
data_end_18='2018-06-01'
first_df=DfTimeFeature(df,data_start_18,data_end_18,svc_incident,site_install,cx_interact)

start_time = pd.to_datetime('2018-06-01')
end_time = pd.to_datetime('2019-06-01')
active_customer=df[(df['TRX_DATE']>start_time)&(df['TRX_DATE']<=end_time)]['CUSTOMER_SITE_ID'].values
first_df["Churn"]=np.where(first_df['CUSTOMER_SITE_ID'].isin(active_customer),0,1)

In [9]:
data_start_19='2015-01-01'
data_end_19='2019-06-01'
second_df=DfTimeFeature(df,data_start_19,data_end_19,svc_incident,site_install,cx_interact)

start_time = pd.to_datetime('2019-06-01')
end_time = pd.to_datetime('2020-06-03')
active_customer=df[(df['TRX_DATE']>start_time)&(df['TRX_DATE']<=end_time)]['CUSTOMER_SITE_ID'].values
second_df["Churn"]=np.where(second_df['CUSTOMER_SITE_ID'].isin(active_customer),0,1)

In [10]:
data_start='2015-01-01'
data_end='2020-06-03'

third_df=DfTimeFeature(df,data_start,data_end,svc_incident,site_install,cx_interact)

In [11]:
all_features=['Frequency','Recency','Unique_Item_Purchased', 'Aggregate_index',
            'PRODUCT_MODEL_mode_Multiple', 'PRODUCT_MODEL_mode_MAKE-UP','PRODUCT_MODEL_mode_SOLVENT',
            'SALES_CHANNEL_mode_Esker','SHORT_VERTICAL_UNKNOWN',
            'Main_Product_mode_CIJ', 'Main_Product_mode_LCM','Main_Product_mode_TIJ', 'Main_Product_mode_TTO',
            'CUSTOMER_CLASS_DISTRIBUTOR', 'CUSTOMER_CLASS_OEM', 
            'NO_OF_SVC_INCIDENTS', 'Contract','Lastest_Amt_Ratio','Lastest_Price_Ratio_Trend',
            'Incident_per_printer', 'Visit_per_TRX','Avg_AMT','Avg_Duration']

In [12]:
tier1_18 = first_df[first_df['Tier']=='Tier 1']
tier2_18 = first_df[first_df['Tier']=='Tier 2']
tier3_18 = first_df[first_df['Tier']=='Tier 3']
#tier4_18 = first_df[first_df['Tier']=='Tier 4']

X1= tier1_18.drop(['Tier',"Churn"],axis=1)[all_features]
X2= tier2_18.drop(['Tier',"Churn"],axis=1)[all_features]
X3= tier3_18.drop(['Tier',"Churn"],axis=1)[all_features]
#X4= tier4_18.drop(['Tier',"Churn"],axis=1)

y1=tier1_18["Churn"]
y2=tier2_18["Churn"]
y3=tier3_18["Churn"]
#y4=tier4_18["Churn"]

In [13]:
tier1_19 = second_df[second_df['Tier']=='Tier 1']
tier2_19 = second_df[second_df['Tier']=='Tier 2']
tier3_19 = second_df[second_df['Tier']=='Tier 3']
#tier4_19 = second_df[second_df['Tier']=='Tier 4']

X1_19= tier1_19.drop(['Tier',"Churn"],axis=1)[all_features]
X2_19= tier2_19.drop(['Tier',"Churn"],axis=1)[all_features]
X3_19= tier3_19.drop(['Tier',"Churn"],axis=1)[all_features]
#X4_19= tier4_19.drop(['Tier',"Churn"],axis=1)

y1_19=tier1_19["Churn"]
y2_19=tier2_19["Churn"]
y3_19=tier3_19["Churn"]
#y4_19=tier4_19["Churn"]

In [14]:
tier1_20 = third_df[third_df['Tier']=='Tier 1']
tier2_20 = third_df[third_df['Tier']=='Tier 2']
tier3_20 = third_df[third_df['Tier']=='Tier 3']
#tier4_20 = third_phase[third_phase['Tier']=='Tier 4']

X1_20= tier1_20.drop(['Tier'],axis=1)[['CUSTOMER_SITE_ID']+all_features]
X2_20= tier2_20.drop(['Tier'],axis=1)[['CUSTOMER_SITE_ID']+all_features]
X3_20= tier3_20.drop(['Tier'],axis=1)[['CUSTOMER_SITE_ID']+all_features]
#X4_20= tier4_20.drop(['Tier'],axis=1)